# Segment Anything with transformers

source: https://huggingface.co/docs/transformers/model_doc/sam 


#WARNING THIS MODEL IS >2,5 Gb! 

In [2]:
#Helper script to check your free disk space
import shutil

def check_disk_space(path="/"):
    total, used, free = shutil.disk_usage(path)
    
    # Convert bytes to gigabytes
    total_gb = total // (2**30)
    used_gb = used // (2**30)
    free_gb = free // (2**30)
    
    print(f"Total disk space: {total_gb} GB")
    print(f"Used disk space: {used_gb} GB")
    print(f"Free disk space: {free_gb} GB")
    print(f"Percentage used: {used * 100 // total}%")

# Check disk space for the root directory
check_disk_space()

Total disk space: 233 GB
Used disk space: 228 GB
Free disk space: 5 GB
Percentage used: 97%


In [1]:
import torch
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
input_points = [[[450, 600]]]  # 2D location of a window in the image

inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu()
)
scores = outputs.iou_scores

/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2D89372E-C77C-3351-8455-B2E540126864> /Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <A84DFEFF-287E-3B94-A7DB-731FA5F9CBBC> /Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-10-02 16:48:51.226625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in

config.json:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [3]:
scores

tensor([[[0.9910, 0.9818, 0.9667]]])

In [10]:
#Code example to show the path where Huggingface stores the models
#Click on link to open it up (works on Appple)
from transformers import file_utils

print(file_utils.default_cache_path)

/Users/michielbontenbal/.cache/huggingface/hub
